In [ ]:
# transformers 완전 삭제 + 폴더까지 수동 제거
!pip uninstall -y transformers
!rm -rf /usr/local/lib/python3.11/dist-packages/transformers*
!rm -rf /usr/local/lib/python3.11/dist-packages/transformers-*

# 안정된 버전 재설치
!pip install transformers==4.40.1

# 강제 런타임 재시작 (이게 핵심)
import os
os.kill(os.getpid(), 9)

Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Successfully uninstalled transformers-4.40.1
  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.


In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, BertForSequenceClassification
print("✅ transformers 작동 확인됨")

✅ transformers 작동 확인됨


In [ ]:
# JSON 파일 수동 업로드
from google.colab import files
uploaded = files.upload()


Saving a.json to a.json


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
df = pd.read_json("a.json")
df = df.rename(columns=lambda x: x.strip())

unsmile_labels = [col for col in df.columns if col != "문장"]
num_labels = len(unsmile_labels)

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({'train': dataset['train'], 'valid': dataset['test']})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess_function(examples):
    inputs = tokenizer(examples["문장"], truncation=True, padding="max_length", max_length=128)
    # Correctly extract labels for batched input and convert to float
    labels = [[float(examples[label][i]) for label in unsmile_labels] for i in range(len(examples["문장"]))]
    inputs["labels"] = labels
    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/13504 [00:00<?, ? examples/s]

Map:   0%|          | 0/1501 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    preds = (pred.predictions > 0.5).astype(int)
    labels = pred.label_ids
    return {
        'f1': f1_score(labels, preds, average='macro', zero_division=0),
        'precision': precision_score(labels, preds, average='macro', zero_division=0),
        'recall': recall_score(labels, preds, average='macro', zero_division=0),
    }


In [ ]:
training_args = TrainingArguments(

    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-3434445962.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jecool0523 (jecool) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.246200,0.244807,0.000000,0.000000,0.000000
2,0.197300,0.193698,0.295386,0.517616,0.224989
3,0.179700,0.180180,0.428437,0.664149,0.345338


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=633, training_loss=0.23582208961672127, metrics={'train_runtime': 946.9609, 'train_samples_per_second': 42.781, 'train_steps_per_second': 0.668, 'total_flos': 2665028029317120.0, 'train_loss': 0.23582208961672127, 'epoch': 3.0})

In [ ]:
preds = trainer.predict(encoded_dataset["valid"])
pred_labels = (preds.predictions > 0.5).astype(int)

for i in range(5):
    print("문장:", dataset["valid"][i]["문장"])
    print("예측 라벨:", {label: int(pred_labels[i][j]) for j, label in enumerate(unsmile_labels)})
    print("실제 라벨:", {label: dataset["valid"][i][label] for label in unsmile_labels})
    print()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


문장: 불교만 해도 국사책만 들춰봐도 신라 고려시대의 사원의 수탈이 나오는 뎁쇼? 걍 종교는 다 병맛 같은 구석이 있습니다.
예측 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 0, '연령': 0, '지역': 0, '종교': 1, '기타 혐오': 0, '악플/욕설': 0, 'clean': 0, '개인지칭': 0, '장애': 0}
실제 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 0, '연령': 0, '지역': 0, '종교': 1, '기타 혐오': 0, '악플/욕설': 0, 'clean': 0, '개인지칭': 0, '장애': 0}

문장: 동남아 새끼들은 씨를말려야되 좆같은 새끼들
예측 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 1, '연령': 0, '지역': 0, '종교': 0, '기타 혐오': 0, '악플/욕설': 0, 'clean': 0, '개인지칭': 0, '장애': 0}
실제 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 1, '연령': 0, '지역': 0, '종교': 0, '기타 혐오': 0, '악플/욕설': 0, 'clean': 0, '개인지칭': 0, '장애': 0}

문장: 꼭 뒤지세요. 기도드립니다. 나무관셈보살.
예측 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 0, '연령': 0, '지역': 0, '종교': 0, '기타 혐오': 0, '악플/욕설': 0, 'clean': 0, '개인지칭': 0, '장애': 0}
실제 라벨: {'여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 0, '연령': 0, '지역': 0, '종교': 0, '기타 혐오': 0, '악플/욕설': 1, 'clean': 0, '개인지칭': 0, '장애': 0}

문장: 진짜 궁예씨ㅏ박ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
예측 라벨: {'여성/가족': 0, '남성': 0,

In [ ]:
eval_result = trainer.evaluate()
print(eval_result)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.18017975986003876, 'eval_f1': 0.428436719965593, 'eval_precision': 0.6641486380139339, 'eval_recall': 0.34533823995574103, 'eval_runtime': 11.9421, 'eval_samples_per_second': 125.69, 'eval_steps_per_second': 7.871, 'epoch': 3.0}


In [ ]:
def apply_disability_rule(pred_labels, input_texts, label_list):
    """'장애'라는 단어가 있으면 clean=0, 장애=1로 보정"""
    clean_idx = label_list.index("clean")
    dis_idx = label_list.index("장애")

    new_preds = []
    for preds, text in zip(pred_labels, input_texts):
        preds = preds.copy()  # 기존 예측 유지
        if "장애" in text:
            preds[clean_idx] = 0
            preds[dis_idx] = 1
        new_preds.append(preds)
    return new_preds


In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification

model_path = "./results"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving di.csv to di.csv


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, BertForSequenceClassification
import json

In [ ]:
# 1. CSV 불러오기
df = pd.read_csv("di.csv")

# 라벨 순서 반드시 학습할 때 사용한 순서와 동일하게 설정
unsmile_labels = ["여성/가족", "남성", "성소수자", "인종/국적", "연령", "지역", "종교", "기타 혐오", "악플/욕설", "clean", "개인지칭", "장애"]

# 3. 모델 로드
model_path = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# 4. 확률 계산 함수
def get_probabilities(text):
    try:X
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
        return {label: float(round(prob, 4)) for label, prob in zip(unsmile_labels, probs)}
    except:
        return {label: None for label in unsmile_labels}

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './results'.

In [ ]:
  # 5. 예측 적용
df["글"] = df["제목"].fillna("") + " " + df["내용"].fillna("")
df["확률_딕셔너리"] = df["글"].apply(get_probabilities)

# 6. 확률 분해해서 개별 컬럼으로 추가
prob_df = df["확률_딕셔너리"].apply(pd.Series)
df = pd.concat([df.drop(columns=["확률_딕셔너리"]), prob_df], axis=1)

# 7. JSON으로 저장
df.to_json("tndnjswjdqh.json", orient="records", force_ascii=False, indent=2)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
from google.colab import files
files.download("tndnjswjdqh.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>